# Testing your installation

<hr/>

If you are following along on your own machine, running this notebook will serve as a check of your installation. You should double check to make sure this notebook was launched from within the `bebi103` environment.

The code cell below does the necessary setup for Google Colab if you are running this notebook there. To access the notebook using Colab, use [this link](https://colab.research.google.com/github/justinbois/learnbayes-livecode/blob/main/test_setup.ipynb). Note that the cell below will take roughly five minutes to run because it is installing Stan and some other packages.

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet datashader bebi103 arviz cmdstanpy watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    import cmdstanpy; cmdstanpy.install_cmdstan()

By running the cell below. You should see compilation of a Stan model, generation of samples, and finally a corner plot.

In [2]:
import numpy as np

import cmdstanpy
import arviz as az

import bebi103

import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

schools_data = {
    "J": 8,
    "y": [28, 8, -3, 7, -1, 1, 18, 12],
    "sigma": [15, 10, 16, 11, 9, 11, 10, 18],
}

schools_code = """
data {
  int<lower=0> J; // number of schools
  vector[J] y; // estimated treatment effects
  vector<lower=0>[J] sigma; // s.e. of effect estimates
}

parameters {
  real mu;
  real<lower=0> tau;
  vector[J] eta;
}

transformed parameters {
  vector[J] theta = mu + tau * eta;
}

model {
  eta ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
"""

with open("schools_code.stan", "w") as f:
    f.write(schools_code)

sm = cmdstanpy.CmdStanModel(stan_file="schools_code.stan")
samples = sm.sample(data=schools_data, output_dir="./", show_progress=False)
samples = az.from_cmdstanpy(samples)

# Make a corner plot of samples
p = bebi103.viz.corner(samples, parameters=['mu', 'tau'])

bokeh.io.show(p)

Loading BokehJS ...

21:55:12 - cmdstanpy - INFO - compiling stan file /Users/bois/Dropbox/git/learnbayes-livecode/schools_code.stan to exe file /Users/bois/Dropbox/git/learnbayes-livecode/schools_code
21:55:25 - cmdstanpy - INFO - compiled model executable: /Users/bois/Dropbox/git/learnbayes-livecode/schools_code
21:55:25 - cmdstanpy - INFO - CmdStan start processing
21:55:25 - cmdstanpy - INFO - Chain [1] start processing
21:55:25 - cmdstanpy - INFO - Chain [2] start processing
21:55:25 - cmdstanpy - INFO - Chain [3] start processing
21:55:25 - cmdstanpy - INFO - Chain [4] start processing
21:55:26 - cmdstanpy - INFO - Chain [4] done processing
21:55:26 - cmdstanpy - INFO - Chain [3] done processing
21:55:26 - cmdstanpy - INFO - Chain [2] done processing
21:55:26 - cmdstanpy - INFO - Chain [1] done processing


## Computing environment

In [3]:
%load_ext watermark
%watermark -v -p numpy,bokeh,cmdstanpy,arviz,bebi103,jupyterlab
print("CmdStan : {0:d}.{1:d}".format(*cmdstanpy.cmdstan_version()))

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.12.0

numpy     : 1.24.3
bokeh     : 3.2.1
cmdstanpy : 1.1.0
arviz     : 0.16.1
bebi103   : 0.1.14
jupyterlab: 4.0.3

CmdStan : 2.32
